# Working With SQLite & Win32 In Python
In this tutorial we will cover how to import and export data directly from Excel into a databse that we create with SQLite the goal of this tutorial is to introduce how we might dynamically add data to our database.

In [ ]:
# import the libraries
import sqlite3
import win32com.client as win32

## Create the Database & the Table
We will create an in-memory database so that way we aren't creating any files in this tutorial. After that we will need to add a table to our database so we can store the data in it.

In [76]:
# create a memort database & a cursor to control our queries.
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# create the table to store the data.
cursor.execute("""CREATE TABLE people (
                  name TEXT,
                  age INTEGER,
                  salary INTEGER,
                  gender TEXT
                  )""")

<div class="alert alert-block alert-info">
<b>Tip:</b> You can also use standard database files and the steps will be the same.
</div>

## Define the Queries to Pull and Insert the Data
We will need to help functions in order for us to put and pull data to and from the database. The first function is used to query data from the database, its a simple query and will only let us filter the data by age. The second function, is used to insert data into the database, it takes 4 parameters all of which are columns in our table. We use place holders to help format our query and make our function dynamic.

In [ ]:
# define a query to retrieve the data
def get_by_age(age):
    cursor.execute("SELECT * FROM people WHERE age > :desiredAge",{'desiredAge': age})
    return cursor.fetchall()

# define a query to insert the person
def insert_person(name, age, salary, gender):
    with conn:
         cursor.execute("INSERT INTO people VALUES (:name, :age, :salary, :gender)",\
                        {'name':name,'age':age,'salary':salary,'gender':gender})

## Get the Data From Excel
Now that we have the database all taken care of let's get the data from our workbook. All of this should look familair if you've seen the other tutorials, but I did decide to add some variety where we make the code to handle a dynamic range by leveraging the xlToRight & xlDown constants which will allow us to find the last column & row in our data table. Once I get my necessary info, I define the range and store the values in a variable called `raw_data`.

In [84]:
# create our instance of Excel using Early Binding
ExcelApp = win32.gencache.EnsureDispatch("Excel.Application")

# Get the Workbook
ExcelWrkBook = ExcelApp.ActiveWorkbook

# Get the Sheet
ExcelWrkSht = ExcelWrkBook.ActiveSheet

# Get xlConstants
xlRight = win32.constants.xlToRight
xlDown = win32.constants.xlDown

# Get the last row and column
LastCol = ExcelWrkSht.Range("H1").End(xlRight).Column
LastRow = ExcelWrkSht.Range("H1").End(xlDown).Row

# define those cells
FirstCell = ExcelWrkSht.Cells(1, 8)
LastCell = ExcelWrkSht.Cells(LastRow, LastCol)

# get the data
raw_data = ExcelWrkSht.Range(FirstCell, LastCell).Value
raw_data

(('Name', 'Age', 'Salary', 'Gender'),
 ('Alex', 18.0, Decimal('27035'), 'M'),
 ('Mike', 36.0, Decimal('34536'), 'M'),
 ('Sara', 59.0, Decimal('119850'), 'F'),
 ('Josh', 33.0, Decimal('151446'), 'M'),
 ('Marshal', 53.0, Decimal('128722'), 'M'),
 ('Sue', 81.0, Decimal('147950'), 'F'),
 ('David', 38.0, Decimal('71273'), 'M'),
 ('Tony', 30.0, Decimal('62963'), 'M'),
 ('Stephanie', 35.0, Decimal('39199'), 'F'),
 ('Mary', 63.0, Decimal('69612'), 'F'))

<div class="alert alert-block alert-info">
<b>Tip:</b> Remember that the `Value` property returns a tuple of tuples.
</div>

## Load the Data
After we grab the data, we will loop through each of the rows and pull the corresponding info. Keep in mind, I skipped the first row because that is just the column headers. After that, I a ran a new query to pull all the people with an age greater than 10. Both the insert & and the query are leveraging the functions we defined up above.

In [79]:
# upload the data into the database
for row in raw_data[1:]:
    
    # get the details from each row
    name = row[0]
    age = row[1]
    salary = int(row[2])
    gender = row[3]
    
    # insert the person
    insert_person(name, age, salary, gender)

# pull the newly inserted data.
people = get_by_age(10)
people[0:5]

[('Alex', 73, 38921, 'M'),
 ('Mike', 68, 109294, 'M'),
 ('Sara', 50, 124908, 'F'),
 ('Josh', 51, 83313, 'M'),
 ('Marshal', 78, 64480, 'M')]

## Dump the Data Into Excel
With our query complete, we can now drop the data into our workbook. To make sure we can handle queries that return different number of rows. I first define the length of a row of data in our query by selecting the first row and counting the number of elements in it. For the length of the column I simply count the length of the list of data. Once I do this, I define the first and last cell of the range then drop the data into Excel.

In [81]:
# get the length of a row
RowLength = len(people[0])
display(RowLength)

# get the length of the column
ColLength = len(people)
display(ColLength)

# row, column
FirstCell = ExcelWrkSht.Cells(2, 1)
LastCell = ExcelWrkSht.Cells(1 + ColLength, RowLength)

# Get the Range
ExcelRng = ExcelWrkSht.Range(FirstCell, LastCell )

ExcelRng.Value = people
people

4

10

[('Alex', 73, 38921, 'M'),
 ('Mike', 68, 109294, 'M'),
 ('Sara', 50, 124908, 'F'),
 ('Josh', 51, 83313, 'M'),
 ('Marshal', 78, 64480, 'M'),
 ('Sue', 23, 168095, 'F'),
 ('David', 17, 146569, 'M'),
 ('Tony', 58, 55714, 'M'),
 ('Stephanie', 63, 175390, 'F'),
 ('Mary', 63, 176367, 'F')]